In [1]:
import pandas as pd
import os
from collections import Counter, defaultdict
import json
import itertools
import numpy as np
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

import pickle as pkl

/home/srtoner/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
suffix = "small"
# +
with open('embedding_data_final.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)
# -


embed_df = pd.DataFrame(embed)
data = U.load_file(f'data_vF_{suffix}.pkl', 'pkl', config['DATADIR'])
data_df = pd.DataFrame(data)

In [3]:
embed_df = embed_df.merge(data_df, how= 'left', left_on= 'passage_key', right_on = 'passage_key')
embed_df = embed_df.dropna(subset=['author_id', 'sent_embedding'])

In [8]:
embed_df.author_id.unique()

array([  735.,   406.,   972.,  1082.,  5722.,  5101.,   562.,  2881.,
        2174., 16731.])

In [4]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import OneHotEncoder
label_encoder=OneHotEncoder(sparse_output=False)

y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))

n_classes = embed_df.author_id.nunique()

In [13]:

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score

# Split data into train/validation/test sets with a ratio of 70/15/15
train, test = U.train_test_split(data, test_size=0.3, random_state=42)
val, test = U.train_test_split(test, test_size=0.5, random_state=42)

# Preprocess text data - remove stop_words
vectorizer = CountVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(train['text_processed'])
X_val = vectorizer.transform(val['text_processed'])
X_test = vectorizer.transform(test['text_processed'])

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, stop_words="english"
)

AttributeError: module 'lda' has no attribute 'LDA'

None


In [14]:

# Train LDA model
lda = LatentDirichletAllocation(n_components=10) 
                                #,max_iter=50, learning_method='online', random_state=42
lda.fit(X_train)

# Predict subject for documents in validation set
y_val = lda.transform(X_val)

# Predict subject for documents in test set
y_test = lda.transform(X_test)


KeyError: 'subject'

In [15]:

# Convert subject column to numerical labels
labels_train = pd.factorize(train['topic'])
labels_val = pd.factorize(val['topic'])
labels_test = pd.factorize(test['topic'])


# This is probably a better way of getting the classifier
# accuracy_score(labels_val[0], np.apply_along_axis(np.argmax, 1, y_val))


In [16]:
y_train = lda.transform(X_train)

In [17]:
y_index_train = np.apply_along_axis(np.argmax, 1, y_train)
y_index_val = np.apply_along_axis(np.argmax, 1, y_val)

y_lda_topics_train = [subjects[idx] for idx in y_index_train]
y_lda_topics_val = [subjects[idx] for idx in y_index_val]

In [18]:
y_lda_topics_train
y_lda_topics_val

['Short stories',
 'Juvenile fiction',
 'United States',
 'Short stories',
 'Poetry',
 'Short stories',
 'Adventure stories',
 'Juvenile fiction',
 'Poetry',
 'Juvenile fiction',
 'Juvenile fiction',
 'United States',
 'Short stories',
 'Short stories',
 'History',
 'Juvenile fiction',
 'Juvenile fiction',
 'Fiction',
 'Poetry',
 'United States',
 'Poetry',
 '19th century',
 'Short stories',
 'United States',
 'Poetry',
 'Juvenile fiction',
 'Adventure stories',
 'Fiction',
 'Short stories',
 'Juvenile fiction',
 'Fiction',
 'Short stories',
 'Short stories',
 'Juvenile fiction',
 'Short stories',
 'Poetry',
 'Juvenile fiction',
 '19th century',
 'Poetry',
 'Poetry',
 'Translations into English',
 'Fiction',
 'Juvenile fiction',
 'Short stories',
 '19th century',
 'History',
 '19th century',
 'Juvenile fiction',
 'Juvenile fiction',
 'United States',
 'Juvenile fiction',
 'Poetry',
 'Short stories',
 'Translations into English',
 'United States',
 'Poetry',
 'Short stories',
 'Translat

In [1]:
X_train.todense()

NameError: name 'X_train' is not defined

In [20]:


np.mean(train['topic'] == y_lda_topics_train)

0.05738029610312443

In [21]:
labels_val

(array([0, 1, 2, ..., 0, 0, 5]),
 Index(['Fiction', 'Juvenile fiction', 'Description and travel', 'Drama',
        'Short stories', 'Poetry', 'World War', 'Biography',
        'Detective and mystery stories', '19th century',
        'Translations into English', 'History', 'Juvenile literature',
        'Science fiction', 'English', 'Great Britain', 'Children', 'France',
        'Conduct of life', 'Adventure stories', 'American', 'United States',
        'English wit and humor', 'England', 'Social life and customs',
        'Love stories', 'Historical fiction', 'Young women',
        'Personal narratives'],
       dtype='object'))

In [77]:
pred_val

array([0, 0, 0, ..., 0, 0, 0])

In [22]:

# Train a classifier on the LDA topics
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(y_val, labels_val)

# Evaluate performance on the validation set
pred_val = clf.predict(y_val)
acc_val = accuracy_score(labels_val, pred_val)
print('Accuracy on validation set:', acc_val)

# Evaluate performance on the test set
pred_test = clf.predict(y_test)
acc_test = accuracy_score(labels_test, pred_test)
print('Accuracy on test set:', acc_test)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.